
### Supervised Learning
### Activity: Building a Student Intervention System

### Question 1 - Classification vs. Regression
*Your goal for this project is to identify students who might need early intervention before they fail or pass. Which type of supervised learning problem is this, classification or regression? Why?*

Answer:This is a classification problem because only two possibilities are here to occur either the studen will pass or fail.
And the output is descrete in nature to check need of early intervention.

### Question-2
load necessary Python libraries and load the student data. Note that the last column from this dataset, `'passed'`, will be our target label (whether the student graduated or didn't graduate). All other columns are features about each student.

In [207]:
# Import libraries
import pandas as pd
import numpy as np
import seaborn as sns

In [208]:
# Read student data
sdata = pd.read_csv("student-data.csv")

In [209]:
sdata.head(10)

,school,sex,age,address,famsize,Pstatus,Medu,Fedu,Mjob,Fjob,...,internet,romantic,famrel,freetime,goout,Dalc,Walc,health,absences,passed
0,GP,F,18,U,GT3,A,4,4,at_home,teacher,...,no,no,4,3,4,1,1,3,6,no
1,GP,F,17,U,GT3,T,1,1,at_home,other,...,yes,no,5,3,3,1,1,3,4,no
2,GP,F,15,U,LE3,T,1,1,at_home,other,...,yes,no,4,3,2,2,3,3,10,yes
3,GP,F,15,U,GT3,T,4,2,health,services,...,yes,yes,3,2,2,1,1,5,2,yes
4,GP,F,16,U,GT3,T,3,3,other,other,...,no,no,4,3,2,1,2,5,4,yes
5,GP,M,16,U,LE3,T,4,3,services,other,...,yes,no,5,4,2,1,2,5,10,yes
6,GP,M,16,U,LE3,T,2,2,other,other,...,yes,no,4,4,4,1,1,3,0,yes
7,GP,F,17,U,GT3,A,4,4,other,teacher,...,no,no,4,1,4,1,1,1,6,no
8,GP,M,15,U,LE3,A,3,2,services,other,...,yes,no,4,2,2,1,1,1,0,yes
9,GP,M,15,U,GT3,T,3,4,other,other,...,yes,no,5,5,1,1,1,5,0,yes


In [210]:
sdata.shape

(395, 31)

### Question-3
Let's begin by investigating the dataset to determine how many students we have information on, and learn about the graduation rate among these students. In the code cell below, you will need to compute the following:
- The total number of students, `n_students`.
- The total number of features for each student, `n_features`.
- The number of those students who passed, `n_passed`.
- The number of those students who failed, `n_failed`.
- The graduation rate of the class, `grad_rate`, in percent (%).


In [211]:
# Calculate number of students
n_students = len(sdata.index)
n_students

395

In [212]:
# Calculate number of features
n_features = sdata.shape[1] - 1
n_features

30

In [213]:
# Calculate passing students
sdata['passed'].value_counts()

yes    265
no     130
Name: passed, dtype: int64

In [214]:
n_passed = len(sdata['passed'][sdata['passed'] == 'yes'])
n_passed

265

In [215]:
# Calculate failing students
n_failed = len(sdata['passed'][sdata['passed'] == 'no'])
n_failed

130

In [216]:
# Calculate graduation rate
grad_rate = len(sdata['passed'][sdata['passed'] == 'yes'])/len(sdata.index)*100
grad_rate

67.08860759493672

In [217]:
# Print the results
print ("Total number of students:",format(n_students))
print ("Number of features: ",format(n_features))
print ("Number of students who passed:",format(n_passed))
print ("Number of students who failed:",format(n_failed))
print ("Graduation rate of the class:",format(grad_rate))

Total number of students: 395
Number of features:  30
Number of students who passed: 265
Number of students who failed: 130
Graduation rate of the class: 67.08860759493672


## Preparing the Data
you will prepare the data for modeling, training and testing.

### Question-4 Identify feature and target columns


separate the student data into feature and target columns to see if any features are non-numeric.

In [218]:
# Extract feature columns

In [219]:
feature_columns = sdata.columns[:-1]
feature_columns

Index(['school', 'sex', 'age', 'address', 'famsize', 'Pstatus', 'Medu', 'Fedu',
       'Mjob', 'Fjob', 'reason', 'guardian', 'traveltime', 'studytime',
       'failures', 'schoolsup', 'famsup', 'paid', 'activities', 'nursery',
       'higher', 'internet', 'romantic', 'famrel', 'freetime', 'goout', 'Dalc',
       'Walc', 'health', 'absences'],
      dtype='object')

In [220]:
# Extract target column 'passed'

In [221]:
target_column = sdata.columns[-1]
target_column

'passed'

In [222]:
# Separate the data into feature data and target data (X and y, respectively)

In [223]:
X = sdata[feature_columns]
y = sdata[target_column]

In [224]:
X
y
print("\nFeature values:")
print(X.head())


Feature values:
  school sex  age address famsize Pstatus  Medu  Fedu     Mjob      Fjob  ...  \
0     GP   F   18       U     GT3       A     4     4  at_home   teacher  ...   
1     GP   F   17       U     GT3       T     1     1  at_home     other  ...   
2     GP   F   15       U     LE3       T     1     1  at_home     other  ...   
3     GP   F   15       U     GT3       T     4     2   health  services  ...   
4     GP   F   16       U     GT3       T     3     3    other     other  ...   

  higher internet  romantic  famrel  freetime goout Dalc Walc health absences  
0    yes       no        no       4         3     4    1    1      3        6  
1    yes      yes        no       5         3     3    1    1      3        4  
2    yes      yes        no       4         3     2    2    3      3       10  
3    yes      yes       yes       3         2     2    1    1      5        2  
4    yes       no        no       4         3     2    1    2      5        4  

[5 rows x 30 co

### Question-5 Preprocess Feature Columns

As you can see, there are several non-numeric columns that need to be converted! Many of them are simply `yes`/`no`, e.g. `internet`. These can be reasonably converted into `1`/`0` (binary) values.

Other columns, like `Mjob` and `Fjob`, have more than two values, and are known as _categorical variables_. The recommended way to handle such a column is to create as many columns as possible values (e.g. `Fjob_teacher`, `Fjob_other`, `Fjob_services`, etc.), and assign a `1` to one of them and `0` to all others.

These generated columns are sometimes called _dummy variables_, and we will use the [`pandas.get_dummies()`](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.get_dummies.html?highlight=get_dummies#pandas.get_dummies) function to perform this transformation. Run the code cell below to perform the preprocessing routine discussed in this section.

In [225]:
from sklearn.preprocessing import add_dummy_feature

In [226]:
sdata.dtypes.value_counts()

object    18
int64     13
dtype: int64

In [227]:
X = pd.get_dummies(X)
X

,age,Medu,Fedu,traveltime,studytime,failures,famrel,freetime,goout,Dalc,...,activities_no,activities_yes,nursery_no,nursery_yes,higher_no,higher_yes,internet_no,internet_yes,romantic_no,romantic_yes
0,18,4,4,2,2,0,4,3,4,1,...,1,0,0,1,0,1,1,0,1,0
1,17,1,1,1,2,0,5,3,3,1,...,1,0,1,0,0,1,0,1,1,0
2,15,1,1,1,2,3,4,3,2,2,...,1,0,0,1,0,1,0,1,1,0
3,15,4,2,1,3,0,3,2,2,1,...,0,1,0,1,0,1,0,1,0,1
4,16,3,3,1,2,0,4,3,2,1,...,1,0,0,1,0,1,1,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
390,20,2,2,1,2,2,5,5,4,4,...,1,0,0,1,0,1,1,0,1,0
391,17,3,1,2,1,0,2,4,5,3,...,1,0,1,0,0,1,0,1,1,0
392,21,1,1,1,1,3,5,5,3,3,...,1,0,1,0,0,1,1,0,1,0
393,18,3,2,3,1,0,4,4,1,3,...,1,0,1,0,0,1,0,1,1,0


In [228]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
sdata['passed'] = le.fit_transform(sdata['passed'])
sdata

,school,sex,age,address,famsize,Pstatus,Medu,Fedu,Mjob,Fjob,...,internet,romantic,famrel,freetime,goout,Dalc,Walc,health,absences,passed
0,GP,F,18,U,GT3,A,4,4,at_home,teacher,...,no,no,4,3,4,1,1,3,6,0
1,GP,F,17,U,GT3,T,1,1,at_home,other,...,yes,no,5,3,3,1,1,3,4,0
2,GP,F,15,U,LE3,T,1,1,at_home,other,...,yes,no,4,3,2,2,3,3,10,1
3,GP,F,15,U,GT3,T,4,2,health,services,...,yes,yes,3,2,2,1,1,5,2,1
4,GP,F,16,U,GT3,T,3,3,other,other,...,no,no,4,3,2,1,2,5,4,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
390,MS,M,20,U,LE3,A,2,2,services,services,...,no,no,5,5,4,4,5,4,11,0
391,MS,M,17,U,LE3,T,3,1,services,services,...,yes,no,2,4,5,3,4,2,3,1
392,MS,M,21,R,GT3,T,1,1,other,other,...,no,no,5,5,3,3,3,3,3,0
393,MS,M,18,R,LE3,T,3,2,services,other,...,yes,no,4,4,1,3,4,5,0,1


In [229]:
y = sdata['passed']
y

0      0
1      0
2      1
3      1
4      1
      ..
390    0
391    1
392    0
393    1
394    0
Name: passed, Length: 395, dtype: int32

### Question - 6 Implementation: Training and Testing Data Split
So far, we have converted all _categorical_ features into numeric values. For the next step, we split the data (both features and corresponding labels) into training and test sets. you will need to implement the following:
- Randomly shuffle and split the data (`X`, `y`) into training and testing subsets.
  - Use 300 training points (approximately 75%) and 95 testing points (approximately 25%).
  - Set a `random_state` for the function(s) you use, if provided.
  - Store the results in `X_train`, `X_test`, `y_train`, and `y_test`.

In [230]:
# splitting the data into train and test
from sklearn.model_selection import train_test_split
#y = sdata['passed']
#X = sdata.drop(['passed'],axis =1)

In [231]:
X_train,X_test,y_train,y_test = train_test_split(X,y,random_state = 42, test_size = .25)

In [232]:
# Show the results of the split
x_test.count()

school        99
sex           99
age           99
address       99
famsize       99
Pstatus       99
Medu          99
Fedu          99
Mjob          99
Fjob          99
reason        99
guardian      99
traveltime    99
studytime     99
failures      99
schoolsup     99
famsup        99
paid          99
activities    99
nursery       99
higher        99
internet      99
romantic      99
famrel        99
freetime      99
goout         99
Dalc          99
Walc          99
health        99
absences      99
F             99
M             99
dtype: int64

### Question - 7  Training and Evaluating Models
In this section, you will choose 3 supervised learning models that are appropriate for this problem and available in `scikit-learn`. You will first discuss the reasoning behind choosing these three models by considering what you know about the data and each model's strengths and weaknesses. You will then fit the model to varying sizes of training data and measure the accuracy score.

###  Model Application
*List three supervised learning models that are appropriate for this problem. What are the general applications of each model? What are their strengths and weaknesses? Given what you know about the data, why did you choose these models to be applied?*

#explaination
1.**Logistic Regression

General Applications:

Logistic Regression supervised learing and has been applied by Netflix to improve their suggetion technology.

Advantages:

Logistic regression is a linear model used for classification rather than regression.

Disadvantages:

Logistic regression is not accurate when there is a large number of features and/or a large amount of missing data

2.**Support Vector Machine

General Applications:

Support Vector Machines have been used in biological and other sciences. It ias been used to classify proteinds with as many as 90% of proteins classified correctly.

Advantages:

Some advantages of the Support Vector Machines are that they work well for data sets with a high number of features, they are memory efficient, and they are versatile with different kernel types that can be applied in the model.

Disadvantages:

One possible drawback is that the method is likely to struggle if the number of features is much greater than the number of samples.
3.**Decision trees

General Applications:

Has been used in the field of astronomy to filter noise from Hubble Space Telescope images.

Advantages:

Decision trees follows simple, logical decision rules and can hande non-linear decision features.

Disadvantages:

The major drawback of decision trees is that they have a tendency to overfit to the training data and can be less flexible for making predictions on new data sets.

In [233]:
y.head()

0    0
1    0
2    1
3    1
4    1
Name: passed, dtype: int32

In [234]:
# Import the three supervised learning models from sklearn


**Logistic Regression**

In [235]:
# fit model-1  on traning data
#Logistic Regression
from sklearn.linear_model import LogisticRegression
logit_model = LogisticRegression()


In [236]:
logit_model.fit(X_train,y_train)

C:\Users\User\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

In [237]:
# predict on the test data 

In [238]:
y_pred = logit_model.predict(X_test)

In [239]:
# calculate the accuracy score

In [240]:
from sklearn.metrics import confusion_matrix,accuracy_score,precision_score,recall_score,f1_score

In [241]:
print("Accuracy is:",accuracy_score(y_test,y_pred) )
print("Precision_score is:",precision_score(y_test,y_pred,pos_label='positive',average='micro') )
print("Recall_score:",recall_score(y_test,y_pred,pos_label='positive',average='micro') )
print("f1_score is:",f1_score(y_test,y_pred,pos_label='positive',average='micro') )

Accuracy is: 0.7070707070707071
Precision_score is: 0.7070707070707071
Recall_score: 0.7070707070707071
f1_score is: 0.7070707070707071


C:\Users\User\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1295: UserWarning: Note that pos_label (set to 'positive') is ignored when average != 'binary' (got 'micro'). You may use labels=[pos_label] to specify a single positive class.
  warnings.warn("Note that pos_label (set to %r) is ignored when "
C:\Users\User\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1295: UserWarning: Note that pos_label (set to 'positive') is ignored when average != 'binary' (got 'micro'). You may use labels=[pos_label] to specify a single positive class.
  warnings.warn("Note that pos_label (set to %r) is ignored when "
C:\Users\User\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1295: UserWarning: Note that pos_label (set to 'positive') is ignored when average != 'binary' (got 'micro'). You may use labels=[pos_label] to specify a single positive class.
  warnings.warn("Note that pos_label (set to %r) is ignored when "


In [242]:
y_test.value_counts()

1    63
0    36
Name: passed, dtype: int64

In [243]:
confusion_matrix(y_test,y_pred)

array([[13, 23],
       [ 6, 57]], dtype=int64)

In [244]:
# fit the model-2 on traning data and predict on the test data and measure the accuracy

**SVM classifier**

In [245]:
from sklearn.svm import SVC

In [246]:
svm_linear = SVC(kernel ='linear')
svm_linear.fit(X_train,y_train)
y_pred = svm_linear.predict(X_test)

In [247]:
print("Accuracy using linear svm is:",accuracy_score(y_test,y_pred))

Accuracy using linear svm is: 0.6868686868686869


**Decision Tree Classifier**

In [248]:
# fit the model-3 on traning data and predict on the test data and measure the accuracy

In [249]:
from sklearn.tree import DecisionTreeClassifier
#from sklearn import metrics
#from sklearn.metrics import f1_score, confusion_matrix, accuracy_score

In [250]:
dtmodel = DecisionTreeClassifier()

In [251]:
dtmodel.fit(X_train,y_train)

DecisionTreeClassifier()

In [252]:
y_pred = dtmodel.predict(X_test)

In [253]:
print("Accuracy is:",dtmodel.score(X_test,y_pred))

Accuracy is: 1.0


In [254]:
confusion_matrix(y_test,y_pred)

array([[15, 21],
       [25, 38]], dtype=int64)